# Risk Score Analysis - Data Exploration

This notebook explores the datasets used for risk score analysis. We'll examine the structure, distributions, and relationships in the data to inform feature engineering and model development.

In [ ]:
# Import required libraries
import os
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnan, when, count, mean, stddev, min, max

# Configure matplotlib
%matplotlib inline
plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

In [ ]:
# Import project modules
import sys
sys.path.append('..')

from src.config import (
    DELTA_TABLES,
    NUMERICAL_FEATURES,
    CATEGORICAL_FEATURES,
    DATE_FEATURES,
    TARGET_COLUMN
)
from src.utils import get_spark_session, read_delta_table

In [ ]:
# Initialize Spark Session
spark = get_spark_session()
print(f"Spark version: {spark.version}")

## 1. Load Raw Data

Let's load the raw data from Delta tables and explore its structure.

In [ ]:
# Read raw data from Delta tables
applications_df = read_delta_table(spark, f"{DELTA_TABLES['raw_data']}/loan_applications")
customers_df = read_delta_table(spark, f"{DELTA_TABLES['raw_data']}/customers")
performance_df = read_delta_table(spark, f"{DELTA_TABLES['raw_data']}/loan_performance")

# If tables don't exist yet, print a message
if applications_df is None:
    print("Loan applications table not found. Run the data ingestion pipeline first.")
if customers_df is None:
    print("Customers table not found. Run the data ingestion pipeline first.")
if performance_df is None:
    print("Loan performance table not found. Run the data ingestion pipeline first.")

### 1.1 Data Schema and Basic Stats

In [ ]:
# Function to examine dataframe structure and stats
def examine_dataframe(df, name):
    if df is None:
        print(f"{name} dataframe is not available")
        return
        
    print(f"\n{'='*50}")
    print(f"Examining {name} Dataset")
    print(f"{'='*50}")
    
    # Basic info
    print(f"Number of records: {df.count()}")
    print(f"Number of columns: {len(df.columns)}")
    print("\nSchema:")
    df.printSchema()
    
    # Show sample data
    print("\nSample data:")
    df.show(5, truncate=False)
    
    # Calculate null counts
    null_counts = []
    for column in df.columns:
        null_count = df.filter(col(column).isNull() | isnan(column)).count()
        null_percent = (null_count / df.count()) * 100 if df.count() > 0 else 0
        null_counts.append((column, null_count, null_percent))
    
    print("\nNull value analysis:")
    for column, count, percent in null_counts:
        if count > 0:
            print(f"{column}: {count} nulls ({percent:.2f}%)")
            
    return null_counts

In [ ]:
# Examine each dataset
app_nulls = examine_dataframe(applications_df, "Loan Applications")
cust_nulls = examine_dataframe(customers_df, "Customers")
perf_nulls = examine_dataframe(performance_df, "Loan Performance")

### 1.2 Numerical Feature Distributions

In [ ]:
# Function to analyze numerical features
def analyze_numerical_features(df, features, name):
    if df is None:
        print(f"{name} dataframe is not available")
        return
        
    print(f"\n{'='*50}")
    print(f"Numerical Feature Analysis for {name}")
    print(f"{'='*50}")
    
    # Calculate statistics for each numerical feature
    stats = {}
    for feature in features:
        if feature in df.columns:
            stats_df = df.select(
                min(feature).alias("min"),
                max(feature).alias("max"),
                mean(feature).alias("mean"),
                stddev(feature).alias("stddev")
            ).collect()[0]
            
            stats[feature] = {
                "min": stats_df["min"],
                "max": stats_df["max"],
                "mean": stats_df["mean"],
                "stddev": stats_df["stddev"]
            }
    
    # Display statistics
    for feature, stat in stats.items():
        print(f"\nFeature: {feature}")
        for metric, value in stat.items():
            print(f"  {metric}: {value}")
    
    # Convert to Pandas for visualization
    pd_df = df.select([col(f) for f in features if f in df.columns]).toPandas()
    
    # Create histograms for each numerical feature
    fig, axs = plt.subplots(len(pd_df.columns), 1, figsize=(12, 4*len(pd_df.columns)))
    
    for i, feature in enumerate(pd_df.columns):
        if len(pd_df.columns) > 1:
            ax = axs[i]
        else:
            ax = axs
            
        pd_df[feature].hist(bins=30, ax=ax)
        ax.set_title(f"{feature} Distribution")
        ax.set_xlabel(feature)
        ax.set_ylabel("Frequency")
    
    plt.tight_layout()
    plt.show()
    
    return stats, pd_df

In [ ]:
# Define numerical features for each dataset
app_num_features = ["loan_amount", "loan_term", "interest_rate", "annual_income", "debt_to_income_ratio"]
cust_num_features = ["age", "num_credit_lines", "num_late_payments_30d", "num_late_payments_60d", 
                     "num_late_payments_90d", "credit_score", "utilization_rate", "num_inquiries_6m"]
perf_num_features = ["current_balance", "remaining_payments", "days_past_due"]

# Analyze numerical features for each dataset
app_num_stats, app_num_df = analyze_numerical_features(applications_df, app_num_features, "Loan Applications")
cust_num_stats, cust_num_df = analyze_numerical_features(customers_df, cust_num_features, "Customers")
perf_num_stats, perf_num_df = analyze_numerical_features(performance_df, perf_num_features, "Loan Performance")

### 1.3 Categorical Feature Distributions

In [ ]:
# Function to analyze categorical features
def analyze_categorical_features(df, features, name):
    if df is None:
        print(f"{name} dataframe is not available")
        return
        
    print(f"\n{'='*50}")
    print(f"Categorical Feature Analysis for {name}")
    print(f"{'='*50}")
    
    # Calculate value counts for each categorical feature
    value_counts = {}
    
    for feature in features:
        if feature in df.columns:
            counts = df.groupBy(feature).count().orderBy("count", ascending=False).collect()
            value_counts[feature] = [(row[feature], row["count"]) for row in counts]
    
    # Display value counts
    for feature, counts in value_counts.items():
        print(f"\nFeature: {feature}")
        for value, count in counts:
            print(f"  {value}: {count}")
    
    # Create bar plots for visualizing categorical distributions
    for feature, counts in value_counts.items():
        # Convert to pandas for easier plotting
        pd_counts = pd.DataFrame(counts, columns=[feature, 'count'])
        
        # Limit to top 10 categories if there are many
        if len(pd_counts) > 10:
            pd_counts = pd_counts.sort_values('count', ascending=False).head(10)
        
        plt.figure(figsize=(12, 6))
        plt.bar(pd_counts[feature].astype(str), pd_counts['count'])
        plt.title(f"{feature} Distribution (Top 10)")
        plt.xlabel(feature)
        plt.ylabel("Count")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    
    return value_counts

In [ ]:
# Define categorical features for each dataset
app_cat_features = ["loan_purpose", "loan_grade", "employment_status", "home_ownership", 
                    "verification_status", "application_type", "state"]
perf_cat_features = ["loan_status", "default_status"]

# Analyze categorical features for each dataset
app_cat_counts = analyze_categorical_features(applications_df, app_cat_features, "Loan Applications")
perf_cat_counts = analyze_categorical_features(performance_df, perf_cat_features, "Loan Performance")

## 2. Target Variable Analysis

In [ ]:
# Analyze the target variable (default_status)
if performance_df is not None:
    target_counts = performance_df.groupBy(TARGET_COLUMN).count().collect()
    
    print(f"\n{'='*50}")
    print(f"Target Variable Analysis: {TARGET_COLUMN}")
    print(f"{'='*50}")
    
    total = performance_df.count()
    for row in target_counts:
        label = row[TARGET_COLUMN]
        count = row["count"]
        percentage = (count / total) * 100
        print(f"Class {label}: {count} records ({percentage:.2f}%)")
    
    # Convert to pandas for visualization
    pd_target = pd.DataFrame([(row[TARGET_COLUMN], row["count"]) for row in target_counts], 
                           columns=[TARGET_COLUMN, 'count'])
    
    # Plot target distribution
    plt.figure(figsize=(8, 6))
    plt.bar(pd_target[TARGET_COLUMN].astype(str), pd_target['count'])
    plt.title(f"{TARGET_COLUMN} Distribution")
    plt.xlabel(TARGET_COLUMN)
    plt.ylabel("Count")
    plt.tight_layout()
    plt.show()
    
    # Check class imbalance
    if len(target_counts) > 1:
        imbalance_ratio = max([row["count"] for row in target_counts]) / min([row["count"] for row in target_counts])
        print(f"\nClass imbalance ratio: {imbalance_ratio:.2f}")
        
        if imbalance_ratio > 10:
            print("Warning: Severe class imbalance detected. Consider using class weights, oversampling, or undersampling.")
        elif imbalance_ratio > 3:
            print("Note: Moderate class imbalance detected. Consider handling during model training.")
else:
    print("Loan performance data with target variable not available")

## 3. Feature Correlations and Relationships

In [ ]:
# Join datasets for correlation analysis
if applications_df is not None and customers_df is not None and performance_df is not None:
    # Join loan applications with customer data
    joined_df = applications_df.join(
        customers_df,
        on="customer_id",
        how="inner"
    )
    
    # Join with loan performance data
    full_df = joined_df.join(
        performance_df,
        on=["application_id", "customer_id"],
        how="inner"
    )
    
    print(f"Joined dataset has {full_df.count()} records")
    
    # Convert to pandas for correlation analysis
    # Select numerical features and target
    corr_features = [
        "loan_amount", "interest_rate", "annual_income", "debt_to_income_ratio",
        "age", "credit_score", "num_credit_lines", "num_late_payments_30d", 
        "num_late_payments_60d", "num_late_payments_90d", "utilization_rate", 
        TARGET_COLUMN
    ]
    
    # Select only features that exist in the dataframe
    existing_features = [f for f in corr_features if f in full_df.columns]
    
    # Convert to pandas (limit to 10,000 rows for memory efficiency in notebook)
    pd_full = full_df.select(existing_features).limit(10000).toPandas()
    
    # Calculate correlation matrix
    corr_matrix = pd_full.corr()
    
    # Plot correlation heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
    plt.title('Feature Correlation Matrix')
    plt.tight_layout()
    plt.show()
    
    # Feature correlation with target variable
    if TARGET_COLUMN in pd_full.columns:
        target_corr = corr_matrix[TARGET_COLUMN].sort_values(ascending=False)
        
        print(f"\n{'='*50}")
        print(f"Feature Correlations with {TARGET_COLUMN}")
        print(f"{'='*50}")
        print(target_corr)
        
        # Plot top correlated features
        plt.figure(figsize=(10, 8))
        target_corr = target_corr.drop(TARGET_COLUMN)  # Remove self-correlation
        target_corr.plot(kind='barh')
        plt.title(f'Feature Correlation with {TARGET_COLUMN}')
        plt.xlabel('Correlation Coefficient')
        plt.tight_layout()
        plt.show()
else:
    print("One or more required datasets not available for correlation analysis")

## 4. Relationship Between Categorical Features and Target

In [ ]:
# Analyze relationship between categorical features and target
if 'full_df' in locals():
    categorical_features = [
        "loan_purpose", "loan_grade", "employment_status", "home_ownership",
        "verification_status", "application_type", "state"
    ]
    
    # Select only features that exist in the dataframe
    existing_cat_features = [f for f in categorical_features if f in full_df.columns]
    
    print(f"\n{'='*50}")
    print(f"Categorical Features vs {TARGET_COLUMN}")
    print(f"{'='*50}")
    
    for feature in existing_cat_features:
        # Calculate default rate by category
        category_stats = full_df.groupBy(feature).agg(
            count("*").alias("total"),
            sum(TARGET_COLUMN).alias("defaults"),
            (sum(TARGET_COLUMN) / count("*") * 100).alias("default_rate")
        ).orderBy("default_rate", ascending=False)
        
        # Convert to pandas for visualization
        pd_cats = category_stats.toPandas()
        
        print(f"\nFeature: {feature}")
        for _, row in pd_cats.iterrows():
            print(f"  {row[feature]}: {row['default_rate']:.2f}% default rate ({row['defaults']} defaults out of {row['total']} loans)")
        
        # Plot default rate by category
        plt.figure(figsize=(12, 6))
        sns.barplot(x=feature, y='default_rate', data=pd_cats)
        plt.title(f'Default Rate by {feature}')
        plt.ylabel('Default Rate (%)')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
else:
    print("Joined dataset not available for categorical analysis")

## 5. Key Insights and Recommendations

Based on the exploratory data analysis, here are some key insights and recommendations for model development:

### Data Quality
- [Document your observations about missing values]
- [Document your observations about outliers]
- [Document any data quality issues to address]

### Feature Engineering Ideas
- [List feature engineering ideas based on your analysis]
- [Identify potential interaction terms]
- [Suggest transformations for skewed distributions]

### Model Development Recommendations
- [Document class imbalance handling approach]
- [Suggest potential algorithms based on the data characteristics]
- [Identify potential evaluation metrics]

### Next Steps
- Develop a robust feature engineering pipeline
- Implement data cleaning and preparation steps
- Develop and test multiple modeling approaches
- Create a risk scoring system based on model outputs